In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)



Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 1000 points of test data (mnist.test), and 5000 points of validation data (mnist.validation). MNIST data point has two parts. mnist.train.images correspond to mnist.train.labels. 

We flatten the 28x28 matrix into 784 numbers as softmax/logistic regression won't care about the structure. The result is that mnist.train.images is a tensor (an n-dimensional array) with a shape of [55000, 784]. Each entry in the tensor is a pixel intensity between 0 and 1.

Each image in MNIST has a corresponding label, a number between 0 and 9 representing the digit drawn in the image. 

We store the labels as "one-hot vectors" Which means something like 3 would be [0,0,0,1,0,0,0,0,0,0]. Conseuqnetly, mnist.train.labels is a [55000,10] array of floats


Basically, we want to be able to say that some things are more likely independent of the input by adding a bias. The result is that the evidence for class i given an input x is

evidence$_i$ = $\sum_j W_{i, j}x_{j} + b_i$

where $W_i$ is the weights and $b_i$ is the bias for class i, and j is an index for summing over the pixels in our input image x. We then convert the evidence tallies into our predicted probabilities y using the "softmax" function

y = softmax(evidence)

Here softmax is serving as an "activatoin" or "link" function, shaping the output of our linear functoin into the form we want - in this case, a probability distribution over 10 cases. You can think of it as converting tallies of evidence into probabilities of our input being in each class. It's defined as:

softmax(x) = normalize(exp(x))

And once expanded, you attain

$softmax(x)_i = \frac{exp(x_i)}{\sum_j exp(x_j)}$

In [3]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])

#x isn't a specific value, it's a placeholder

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

#W is 784, 10 so when we multiply it by the 784,1 matrix, we create a 
#10x1 matrix of confidence intervals

y = tf.nn.softmax(tf.matmul(x, W) + b)

### Training

In order to train our model, we need to define a cost function. One very common very nice function to determine the loss is called "cross entropy" Cross-entropy arises fro mthinking about information compressing codes in information theory but it winds up being an important idea in a lot of areas, from gambling to machine learning. It's defined as

$H_{y'}(y) = -\sum_i y_{i}^{'}log(y_i)$

where y is our predicted probability distribution, and y' is the true distribution (the one-hot vector with digit labels.) In some rough sense, the cross-entropy is measuring how inefficient our predictions.

To implement cross-entropy we need to first add a new placeholder to input correct answers:


In [4]:
y_ = tf.placeholder(tf.float32, [None, 10])

Then we implement cross entropy function

$-\sum y'log(y)$

In [5]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = [1]))

First, tf.log computes the ogarithm of each element of y. Next, we multiply each element of y_ with the corresponding element of tf.log(y). Then tf.reduce_sum adds the elements in the second dimension of y, due to reduction_indices[1] parameter. Finally, tf.reduce_mean computes the mean over all the examles in the batch. 

Note that in the source code, we don't use this formulation, 

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
sess = tf.InteractiveSession()

In [12]:
tf.global_variables_initializer().run()

In [14]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})